In [4]:
# imports
import sys
import re
from pyhive import hive
import pandas as pd
import seaborn as sns
import numpy as np
import datetime
import requests
import json
import matplotlib.pyplot as plt
import missingno as msno
import warnings
warnings.filterwarnings('ignore')
sys.path.append('/data/datamining/envir/Py_functions')
from get_data.get_data_func import * ##取数function

import success


In [5]:
cough = query_data(sql = 'SELECT * FROM my_dw.dwd_couth_idx_d_old')

In [7]:
cough.to_csv('cough.csv',index = False)

In [5]:
# ## 提取咳嗽标签
# cough = query_data(method ='inceptor', username = 'szhuangdancheng', password = '2020@hdc',
#            sql = '''SELECT date(attack_date) AS `date`,batchno,ffieldid,unit_no
#                     FROM mydw.source_my_sc_recipe AS re 
#                     INNER JOIN mydw.source_my_sc_cfq_appy_sympto neo
#                     ON neo.my_recipeid = re.id
#                     WHERE segmentid = 13906
#                     AND hxd_sympto LIKE '%咳嗽%' AND batchno in ''' + str(batchlist))
# cough['cough'] = 1
# cough = cough.rename({'date':'time','unit_no':'unit'},axis = 1).astype({'time':'datetime64'})
# ## 提取腹泻标签
# diarrhea = query_data(method ='inceptor', username = 'szhuangdancheng', password = '2020@hdc',
#            sql = '''SELECT date(attack_date) AS `date`,batchno,ffieldid,unit_no
#                     FROM mydw.source_my_sc_recipe AS re 
#                     INNER JOIN mydw.source_my_sc_cfq_appy_sympto neo
#                     ON neo.my_recipeid = re.id
#                     WHERE segmentid = 13906
#                     AND fmw_sympto LIKE '%腹泻%' AND batchno in ''' + str(batchlist))
# diarrhea['diarrhea'] = 1
# diarrhea = diarrhea.rename({'date':'time','unit_no':'unit'},axis = 1).astype({'time':'datetime64'})
# ## 合成标签
# cough['month'] = cough.time.dt.month
# cough = cough.groupby(['ffieldid','unit','month','batchno']).sum().reset_index()
# diarrhea['month'] = diarrhea.time.dt.month
# diarrhea = diarrhea.groupby(['ffieldid','unit','month','batchno']).sum().reset_index()
# cough = cough.drop(cough[cough.unit == '保育猪'].index).astype({'unit':'int'})
# diarrhea = diarrhea.drop(diarrhea[diarrhea.unit == '保育猪'].index).astype({'unit':'int'})
# label = cough.merge(diarrhea, on = ['ffieldid','unit','month','batchno'], how = 'outer')

In [7]:
## 融合保存
# shengchan_data = step_2_data.merge(label,on = ['ffieldid','unit','batchno'] , how = 'left')
# shengchan_data = shengchan_data.fillna(0).astype({'month':'int','cough':'int','diarrhea':'int'})
# shengchan_data.to_csv('shengchan.csv',index = False)

In [140]:
## 数据读取并清洗
data = pd.read_csv('shengchan.csv').astype({'ffieldid':'str','unit':'str'})
data.liverate = data.liverate.apply(lambda x : float(x[:-1]))
print('原始数据量:', data.shape[0])
def data_clean(x):
    try:
        if x < 0:
            return np.nan
        else:
            return x
    except:
        return x 
def outlier(data):
    q3 = data.quantile(0.75)
    q1 = data.quantile(0.25)
    upb = q3 + 1.75 * (q3 - q1)
    lrb = q1 - 1.75 * (q3 - q1)
    return upb,lrb    
## drop 负值
data = data.applymap(data_clean).dropna()
print('数据清洗后:',data.shape[0])
## drop 没有咳嗽和腹泻数据的厂区
data_fid = data.groupby('ffieldid')['cough','diarrhea'].sum().reset_index()
bad_lst = list(data_fid.loc[(data_fid.cough == 0) & (data_fid.diarrhea == 0),'ffieldid'])
data = data.drop(data[data.ffieldid.isin(bad_lst)].index)
print('数据清洗后:',data.shape[0])
## drop生产成绩离群点
coln_lst = ['ingoodqty','outqty','indayold','outdayold','staravgweight','endavgweight',
            'liverate','seventydayweight','sixtothirtyfeedrate','dayaddweight']
df_bounds = pd.DataFrame([])
for coln in coln_lst:
    df_bounds[coln] = outlier(data[coln])
for coln in coln_lst:
    data = data.drop((data[(data[coln] >= df_bounds[coln][0])|(data[coln] <= df_bounds[coln][1])]).index)
    print('数据清洗后:',data.shape[0])

原始数据量: 23385
数据清洗后: 23275
数据清洗后: 22749
数据清洗后: 21618
数据清洗后: 21356
数据清洗后: 20554
数据清洗后: 20248
数据清洗后: 20070
数据清洗后: 19857
数据清洗后: 18925
数据清洗后: 18873
数据清洗后: 18394
数据清洗后: 18364


In [137]:
def classify(x):
    if x == 0:
        return x
    if x > 0 and x < 5 :
        return 1
    if x >= 5 :
        return 2 

In [ ]:
data.groupby('month').

In [139]:
data.cough.apply(classify).value_counts()

In [105]:
data.describe()

,ingoodqty,inbadqty,outqty,outbadqty,indayold,outdayold,staravgweight,endavgweight,liverate,seventydayweight,sixtothirtyfeedrate,dayaddweight,month,cough,diarrhea
count,18364.000000,18364.000000,18364.000000,18364.000000,18364.000000,18364.000000,18364.000000,18364.000000,18364.000000,18364.000000,18364.00000,18364.000000,18364.000000,18364.000000,18364.000000
mean,1100.667338,1.796885,1042.496515,2.641745,24.994897,76.124825,6.665444,24.737809,94.845977,21.345113,1.70660,347.653269,5.632760,2.366151,1.122577
std,275.051526,8.443341,263.299623,15.258894,3.481299,10.409264,0.921075,5.373415,3.848036,3.496930,0.20362,62.421003,4.034941,3.630349,2.642045
min,423.000000,0.000000,358.000000,0.000000,13.970000,43.330000,3.860000,9.490000,81.830000,9.840000,1.09000,146.410000,0.000000,0.000000,0.000000
25%,962.000000,0.000000,905.000000,0.000000,22.490000,69.040000,6.020000,20.990000,92.930000,19.110000,1.58000,305.170000,0.000000,0.000000,0.000000
50%,1103.000000,0.000000,1042.000000,0.000000,24.660000,76.090000,6.560000,24.450000,95.990000,21.610000,1.70000,347.720000,8.000000,1.000000,0.000000
75%,1239.000000,0.000000,1182.250000,0.000000,27.090000,83.280000,7.210000,28.030000,97.740000,23.750000,1.83000,388.582500,9.000000,3.000000,1.000000
max,1789.000000,350.000000,1726.000000,623.000000,36.270000,110.990000,9.620000,41.840000,100.000000,32.490000,2.34000,547.410000,12.000000,85.000000,115.000000


In [141]:
data.cough = data.cough.apply(classify)
data.groupby(['cough'])['liverate','dayaddweight'].mean()

,liverate,dayaddweight
cough,,
0,94.685197,338.469955
1,94.904498,355.770910
2,95.081136,350.330214
